1. How many pit stops does the constructor team usually take?
2. How well does the qualifying race predict the winner of the race?
DATA DESCRIPTION: For Question 1, the following tables are needed: pit_stops, results, constructors

Each row in pit_stops represents one pit stop a driver (identified by driverId) makes in a certain race (identified by raceId). The number of stops in this race is indexed by a stop number. Other pertinent information such as the lap, time, duration of this stop is provided. There are 9634 observations.

In [2]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date, time, datetime
import numpy as np

In [3]:
# Import data, read csv files as Pandas Dataframe
pitstops = pd.read_csv("data_raw/pit_stops.csv")
results = pd.read_csv("data_raw/results.csv")
constructors = pd.read_csv("data_raw/constructors.csv")
races = pd.read_csv("data_raw/races.csv")


In [ ]:
# Display data
pitstops.dtypes

In [4]:
# DATA CLEANING: remove null values, convert to int (if needed)
# Inspects pitstops: no null value, convert time to datetime format
pitstops["time"] = pd.to_datetime(pitstops["time"], format = "%H:%M:%S")
pitstops.dtypes

# Inspect results: replace null values, convert column to int (if needed)
results["number"] = results["number"].replace(["\\N"], [np.nan])
results['number'] = pd.to_numeric(results['number'])
results["position"] = results["position"].replace(["\\N"], [np.nan])
results['position'] = pd.to_numeric(results['position'])
results["time"] = results["time"].replace(["\\N"], [np.nan])
results["milliseconds"] = results["milliseconds"].replace(["\\N"], [np.nan])
results['milliseconds'] = pd.to_numeric(results['milliseconds'])
results["fastestLap"] = results["fastestLap"].replace(["\\N"], [np.nan])
results['fastestLap'] = pd.to_numeric(results['fastestLap'])
results["rank"] = results["rank"].replace(["\\N"], [np.nan])
results['rank'] = pd.to_numeric(results['rank'])
results["fastestLapTime"] = results["fastestLapTime"].replace(["\\N"], [np.nan])
results["fastestLapSpeed"] = results["fastestLapSpeed"].replace(["\\N"], [np.nan])
results.dtypes

# Inspect 


# Store cleaned data
pitstops.to_csv("data_clean/pitstops.csv")
results.to_csv("data_clean/results.csv")

Create dataframe: calculations + merges

I. Calculations Pt 1 Selected columns needed from pitstops (driverId, raceId, stop, milliseconds) and found total stops by grouping rows along driverId and raceId and finding the max value of stop. Then renamed stop to total_stop and milliseconds to stop_length. To check the validity of this calculation, we manually sorted values of pitstops to confirm the total number of stops for driver 816, 815, 814. So far, this table shows the number of stops of 4701 drivers per race and the duration of their stop.

II. Merges Merged columns 'constructorId' and 'milliseconds' from results based on 'raceId' and 'driverId' to obtain the constructor and final result for each driver. We renamed 'milliseconds' to 'time' We noted some drivers did not finish and correspondingly, the 'milliseconds' column does not have a value. Thus we dropped each observation where the driver did not complete the race.There are now a total of 2486 observations. Next, we merged 'name' from constructors to obtain the constructor name that each driver is representing for the current race. Lastly, we merged 'year' from races to obtain the year in which the race was held. We noted that the pits stops data contained exclusively the years 2011-2022.

III. Calculations Pt 2 Computed the average number of stops, time, stop length for each constructor of each race using aggregate statistics. The year, raceId, and name of the constructor are kept in this final dataframe by keeping the first value that fits in this group. The final number of observations is 1561. The columns are renamed accordingly as averages. This is then saved as a new csv in the data_final folder.

In [5]:
# Code for Graph 1: 

# I. Calculations Pt 1
g_one = pitstops.loc[pitstops.groupby(['raceId', 'driverId'], sort= True)['stop'].idxmax()][['raceId', 'driverId', 'stop', 'milliseconds']]
g_one = g_one.rename(columns={'stop':'total_stops', 'milliseconds': 'stop_length'})
sort = pitstops.sort_values(by = ['driverId', 'raceId' ]) # check g_one values for total_stop

# II. Merge
g_one = pd.merge(g_one, results[['driverId', 'raceId','constructorId', 'milliseconds']], on=['raceId', 'driverId'], how='left')
g_one = g_one[g_one['milliseconds'].notna()]
g_one = g_one.rename(columns={'milliseconds': 'time'})
g_one = pd.merge(g_one, constructors[['constructorId', 'name']], on='constructorId', how='left')
g_one = pd.merge(g_one, races[['raceId', 'year']], on='raceId', how='left')

# III. Calculations Pt 2
g_one = g_one.groupby(['constructorId', 'raceId'], as_index=True).agg({
    'name': 'first',
    'year': 'first',
    'raceId': 'first',
    'total_stops':'mean',
    'time': 'mean',
    'stop_length': 'mean' 
})
g_one = g_one.rename(columns={'total_stops': 'avg_num_stops', 'time':'avg_time', 'stop_length':'avg_stop_length', 'name':'Constructor'})

g_one.to_csv("data_final/g_one_data.csv")
g_one.describe()

,year,raceId,avg_num_stops,avg_time,avg_stop_length
count,1561.000000,1561.000000,1561.000000,1.561000e+03,1.561000e+03
mean,2016.535554,966.971813,2.095131,6.037321e+06,8.166231e+04
std,3.580957,76.961799,0.952097,1.203749e+06,2.765936e+05
min,2011.000000,841.000000,1.000000,4.526665e+06,1.397300e+04
25%,2013.000000,894.000000,1.000000,5.435111e+06,2.191900e+04
50%,2016.000000,967.000000,2.000000,5.773158e+06,2.363900e+04
75%,2020.000000,1035.000000,3.000000,6.193374e+06,2.630950e+04
max,2022.000000,1096.000000,6.000000,1.472798e+07,2.048175e+06


Results

In [6]:
import plotly.express as px

df = g_one
df['inv_stop_length'] = df['avg_stop_length'].apply(lambda x: (1/x)*10000000)
# The inverse stop length creates a larger bubble for a shorter avg. pit stop duration
# Note the multiplication factor allows the min value (the longest pit stop duration to be greater than 1 for graphing purposes)

fig = px.scatter(df, x="avg_num_stops", y="avg_time", animation_frame="Constructor", animation_group="raceId",
           size="inv_stop_length", color="raceId", hover_name="year",
           range_x=[0, 7], range_y=[4000000, 16000000],
           title= "Number Pit Stops vs Overall Time Per Constructor",
           labels={
            "avg_num_stops": "Average Number of Stops",
            "avg_time": "Average Race Time (ms)"
            },
            trendline='ols'
        )
fig.show()

# Fig 2 used to calculate overall trendline + statistical significance (not graphed)
fig2 = px.scatter(df, x="avg_num_stops", y="avg_time", trendline='ols', trendline_scope="overall")
results = px.get_trendline_results(fig2)
results = results.iloc[0]["px_fit_results"].summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.124
Model:                            OLS   Adj. R-squared:                  0.123
Method:                 Least Squares   F-statistic:                     220.4
Date:                Sat, 22 Apr 2023   Prob (F-statistic):           9.73e-47
Time:                        18:22:50   Log-Likelihood:                -23967.
No. Observations:                1561   AIC:                         4.794e+04
Df Residuals:                    1559   BIC:                         4.795e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.105e+06    6.9e+04     74.017      0.0